In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

In [ ]:
image_dir = '/path/to/image/folder/'
train_labels = pd.read_csv('/path/to/label/folder/train_data_new.csv')
val_labels = pd.read_csv('/path/to/label/folder/val_data_new.csv')
test_labels = pd.read_csv('/path/to/label/folder/test_data_new.csv')

In [ ]:
# Choose one of train val and test
dataframe = val_labels

In [ ]:
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((512, 512))
    img = np.asarray(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [ ]:
which_model = 'universal'

if which_model == 'diagnosis_only':
    model = '/path/to/pretrained/models/7pt_F_diag_T_mgmt_F_exported.h5'
elif which_model == 'universal':
    model = '/path/to/pretrained/models/7pt_T_diag_T_mgmt_T_exported.h5'
else:
    raise NotImplementedError

In [ ]:
classifier = load_model(model)
classifier.trainable = False

# 0 for test and 1 for train phase
print(K.learning_phase()) 

In [ ]:
# Choose clinic or derm
which_modality = 'clinic'

In [ ]:
if which_modality == 'clinic':
    derm_img = np.zeros((1, 512, 512, 3))
elif which_modality == 'derm':
    clinic_img = np.zeros((1, 512, 512, 3))
else:
    raise NotImplementedError
    
meta = np.zeros((1, 1, 1, 14))
aux_input_derm = np.zeros((1, 14, 14, 14))
aux_input_clinic = np.zeros((1, 14, 14, 14))

In [ ]:
if which_modality == 'clinic':
    my_model = tf.keras.Model(classifier.input, classifier.get_layer('vis_out_clinic').output)
elif which_modality == 'derm':
    my_model = tf.keras.Model(classifier.input, classifier.get_layer('vis_out_derm').output)

In [ ]:
for idx in range(len(dataframe)):
    
    img_path = os.path.join(image_dir, dataframe.loc[idx, which_modality])
    img = preprocess_image(img_path)

    if which_modality == 'clinic':
        clinic_img = img
    elif which_modality == 'derm':
        derm_img = img
        
    feature = my_model({'input_1_derm':derm_img, 'input_2_clinic':clinic_img, 'aux_input':meta, 'aux_input_derm':aux_input_derm, 'aux_input_clinic':aux_input_clinic}, training=False)
    
    if idx == 0:
        features_all = feature.numpy().squeeze()
    else:
        features_all = np.vstack((features_all, feature.numpy().squeeze()))
        
    print(idx+1, '/', len(dataframe))

In [ ]:
save_dir = '/path/to/the/features/folder/'

np.save(os.path.join(save_dir, 'features_univ_clinic_val.npy'), features_all)